<a href="https://colab.research.google.com/github/tayfununal/nku_lesson/blob/main/Two_model_merging_for_continuous_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import categorical_crossentropy, mse
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import concatenate


In [ ]:
# Loading original data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshaping the array to 4-dims so that it can work with the Keras API
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Making sure that the values are float so that we can get decimal points after division
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print('Number of images in X_train', X_train.shape[0])
print('Number of images in X_test', X_test.shape[0])

In [ ]:
# Adding epsilon
tf.random.set_seed(8)
epsilon = tf.random.normal((60000,28,28,1), mean=0, stddev= 0.1)
epsilon_test = tf.random.normal((10000,28,28,1), mean=0, stddev= 0.1)

X_train_with_epsilon = tf.math.add(X_train, epsilon)
X_test_with_epsilon = tf.math.add(X_test, epsilon_test)

print('X_train shape:', X_train_with_epsilon.shape)
print('Number of images in X_train', X_train_with_epsilon.shape[0])
print('Number of images in X_test', X_test_with_epsilon.shape[0])

In [ ]:
# Making y_train and y_test to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

feature_input = Input(shape=(28,28))

hidden = Flatten()(feature_input)
hidden = Dense(512, activation="relu")(hidden)
hidden = Dense(512, activation="relu")(hidden)

output1 = Dense(10, activation='softmax',name="categorical_loss")(hidden)

# models with epsilon
feature_input_with_epsilon = Input(shape=(28,28))

hidden2 = Flatten()(feature_input_with_epsilon)
hidden2 = Dense(512, activation="relu")(hidden2)
hidden2 = Dense(512, activation="relu")(hidden2)

output2 = Dense(10, activation='softmax', name="mse")(hidden2)

model = Model(inputs= [feature_input, feature_input_with_epsilon], outputs=[output1, output2])
model.summary()

In [ ]:
# Culculating Custom Loss
model.compile(optimizer="Adam", loss= ['categorical_crossentropy','mse'], loss_weights=[1,2], metrics=["accuracy"])
history = model.fit([X_train, X_train_with_epsilon], [y_train, y_train], epochs=5, verbose=2, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate([X_test, X_test], [y_test, y_test], batch_size=32, verbose=1)

In [ ]:
predictions = model.predict([X_test, X_test])
print(predictions[0])

In [ ]:
 fig, ax = plt.subplots(4,5, figsize=(15,15))

for i in range(0,4):
  for j in range(0,5):
    random_int = np.random.randint(1,10000,1)
    ax[i][j].imshow(X_test[random_int].reshape((28,28)))
    ax[i][j].set_title("Value : {} \nPredicted Value : {}".format(int(np.where(y_test[random_int])[1]),np.argmax(predictions[1][random_int])))

plt.show()